In [12]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [33]:
env=gym.make('CartPole-v0').unwrapped
env.reset()

is_python='inline' in matplotlib.get_backend()
if is_python:
    from IPython import display
    
plt.ion()

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
Transition=namedtuple('Transition',('state','action','next_state','reward'))

class ReplayMemory(object):
    
    def __init__(self,capacity):
        self.capacity=capacity
        self.memory=[]
        self.position=0
        
    def push(self,*args):
        """Saves a transition"""
        if len(self.memory)<self.capacity:
            self.memory.append(None)
        self.memory[self.position]=Transisiton(*args)
        self.position=(self.position+1)%self.capacity
        
    def sample(self,batch_size):
        return random.sample(self.memory,batch_size)
    
    def __len__(self):
        return len(self.memory)    

In [22]:
class DQN(nn.Module):
    
    def __init__(self,h,w,outputs):
        super(DQN,self).__init__()
        self.conv1=nn.Conv2d(3,16,kernel_size=5,strides=2)
        self.bn1=nn.BatchNorm2d(16)
        self.conv2=nn.Conv2d(16,32,kernel_size=5,strides=2)
        self.bn2=nn.BatchNorm2d(32)
        self.conv3=nn.Conv2d(32,32,kernel_size=5,strides=2)
        self.bn3=nn.BatchNorm2d(32)
                             
        #Num of linera input connections depend on output of conv2d layers
                             
        def conv2d_size_out(size,kernel_size=5,strides=2):
            return (size-(kernel_size-1)-1)//strides+1
        convw=conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh=conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size=convw*convh*32
        self.head=nn.Linear(linear_input_size,outputs)
        
    def forward():
        x=F.relu(self.bn1(self.conv1(x)))
        x=F.relu(self.bn2(self.conv2(x)))
        x=F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0),-1))
    

In [35]:
"""resize=T.Compose([T.ToPILImage(),T.Resize(40,interpolation=Image.CUBIC),T.ToTensor()])

def get_cart_location(screen_width):
    world_width=env.x_threshold*2
    scale=screen_width/world_width
    return int(env.state[0]*scale+screen_width/2.0)

def get_screen():
    # gym screen is 400*600*3 or 800*1200*3
    screen=env.render(mode="rgb_array").transpose((2,0,1))
    
    #cart is in the lower half
    _,screen_height,screen_width=screeen.shape
    screen=screen[:,int(screen_height*0.4):int(screen_height*0.8)]
    view_width=int(screen_width*0.6)
    cart_location=get_cart_location(screen_width)
    if cart_location<view_width//2:
        slice_range=slice(view_width)
    elif cart_location>(screen_width-view_width//2):
        slice_range=slice(-view_width,None)
    else:
        slice_range=slice(cart_location-view_width//2,cart_location+view_width//2)
        
    screen=screen[:,:,slice_range]
    screen=np.ascontiguousarray(screen,dtype=np.float32)/255
    screen=torch.from_numpy(screen)
    return resize(screen).unsqueeze(0).to(device)

env.reset()
plt.figure()
plt.imshow(get_screen().cpu().squeeze(0).permute(1,2,0).numpy(),interpolation='none')
plt.title('Example of extracted Screen')
plt.show()"""

'resize=T.Compose([T.ToPILImage(),T.Resize(40,interpolation=Image.CUBIC),T.ToTensor()])\n\ndef get_cart_location(screen_width):\n    world_width=env.x_threshold*2\n    scale=screen_width/world_width\n    return int(env.state[0]*scale+screen_width/2.0)\n\ndef get_screen():\n    # gym screen is 400*600*3 or 800*1200*3\n    screen=env.render(mode="rgb_array").transpose((2,0,1))\n    \n    #cart is in the lower half\n    _,screen_height,screen_width=screeen.shape\n    screen=screen[:,int(screen_height*0.4):int(screen_height*0.8)]\n    view_width=int(screen_width*0.6)\n    cart_location=get_cart_location(screen_width)\n    if cart_location<view_width//2:\n        slice_range=slice(view_width)\n    elif cart_location>(screen_width-view_width//2):\n        slice_range=slice(-view_width,None)\n    else:\n        slice_range=slice(cart_location-view_width//2,cart_location+view_width//2)\n        \n    screen=screen[:,:,slice_range]\n    screen=np.ascontiguousarray(screen,dtype=np.float32)/255\n

In [41]:
env.close()
screen=env.render()


NameError: name 'base' is not defined